In [57]:
import chromedriver_autoinstaller
from selenium import webdriver
import pandas as pd
import re
import math
import time
import os, sys
import requests
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains


In [61]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.220 Whale/1.3.51.7 Safari/537.36'}

In [109]:
def open_chromedriver():
    chrome = chromedriver_autoinstaller.install(os.getcwd())

    options = webdriver.ChromeOptions()
    #options.headless = True

    driver = webdriver.Chrome(options=options)
    return(driver)


In [32]:
def open_popup():
    element=driver.find_element_by_xpath('//*[@id="megaHead2015"]/div[1]/div[2]/a[3]')
    
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()
    
    driver.find_element_by_xpath('//*[@id="megaGnb"]/div[1]/div[2]/a[1]').click()

In [33]:
def change_frame():
    iframe = driver.find_element_by_xpath('//*[@id="ifrm_quick_search"]')
    driver.switch_to.frame(iframe)
    

In [34]:
def get_onclick(xpath_value):
    get_onclick = []
    for i in driver.find_elements_by_xpath(xpath_value):
        get_onclick.append(i.get_attribute('onclick'))
        
    return(get_onclick)

In [35]:
def switch_to_tch():
    driver.find_element_by_xpath('//*[@id="menu_quick02"]').click()

In [36]:
def get_tch_home():
    
    tch_home =[]
    for i in range(2,10):
        value = '//*[@id="imgTec01_%d"]' %i

        driver.execute_script(driver.find_element_by_xpath(value).get_attribute('onclick'))

        time.sleep(0.5)

        tch_onclick=get_onclick('/html/body/div/div[9]/form/table/tbody/tr/td[3]/div[2]/table/tbody/tr/td[1]/table/tbody/tr[4]/td/div/table/tbody/tr[*]/td[*]/a')


        for i in tch_onclick:
            driver.execute_script(i)

            time.sleep(0.5)

            tch_home.extend(get_onclick('//*[@id="layer_quick_teacher02"]/span[2]/a'))
        
    return(tch_home)



In [37]:
def get_page_url(tch_home):
    page_url=[]
    for i in tch_home:
        page_url.append('https://www.megastudy.net%s' %re.split("'",i)[1])
    return(page_url)

In [38]:
def select_lec():
    word_check=[]
    for i in driver.find_elements_by_xpath('//*[@id="tecChrTabArea"]/ul/li[*]'):
        word_check.append(i.text)

    num=word_check.index('단과강좌')+1

    driver.find_element_by_xpath('//*[@id="tecChrTabArea"]/ul/li[%s]' %num).click()

    time.sleep(0.5)

In [39]:
def get_data_1():
    name = []
    title=[]
    link=[]
    if len(driver.find_elements_by_xpath('//*[@id="divChrTabArea"]/ul/li[*]'))==0:

        for i in driver.find_elements_by_xpath('//*[@id="iChrList"]/table[*]/tbody/tr[*]/td[1]/a'):
            name.append(i.text)    

        for i in driver.find_elements_by_xpath('//*[@id="iChrList"]/table[*]/tbody/tr[*]/td[2]/div/p[3]/a'):
            title.append(i.text)
            link.append(i.get_attribute('href'))

        subject = driver.find_element_by_xpath('//*[@id="container"]/div[1]/div[1]/h2/a[1]/span').text
        subject = [subject]*len(title)
    else :
        for j in driver.find_elements_by_xpath('//*[@id="divChrTabArea"]/ul/li[*]'):
            driver.execute_script(j.find_element_by_tag_name('a').get_attribute('onclick'))

            time.sleep(0.5)

            for i in driver.find_elements_by_xpath('//*[@id="iChrList"]/table[*]/tbody/tr[*]/td[1]/a'):
                name.append(i.text)    

            for i in driver.find_elements_by_xpath('//*[@id="iChrList"]/table[*]/tbody/tr[*]/td[2]/div/p[3]/a'):
                title.append(i.text)
                link.append(i.get_attribute('href'))

            subject = driver.find_element_by_xpath('//*[@id="container"]/div[1]/div[1]/h2/a[1]/span').text
            subject = [subject]*len(title)
            
    return(name, title, link)

        

In [40]:
def get_data_2(link):
    data_2=[]
    num= 0
    for i in link:
        print(i)
        num=num+1
        print(num,'/',len(link))
        
        table=pd.read_html(i)
        for t in table:
            if list(t)==['강의명','맛보기']:
                sample=t.iloc[:,0]
                break
        list1=list(range(1,len(sample)+1))
        list2= list(sample)
        
        try:
            data_2.append([','.join(['(' + str(a) + ', ' + b+')' for a,b in zip(list1,list2)])])
        except:
            index_value= list(pd.isnull(list2)).index(True)
            list2.pop(index_value)
            list2.pop(index_value)
            
            data_2.append([','.join(['(' + str(a) + ', ' + b+')' for a,b in zip(list1,list2)])])
    
    return(data_2)

In [159]:
def get_sub(lec_link):
    sub=[]
    num=0
    for i in lec_link:
        num=num+1
        
        print(num, '/', len(lec_link))
        
        res=requests.get(i,headers=header)
        soup=BeautifulSoup(res.content, 'html.parser',from_encoding='cp949')
        value=soup.find('p',{'class':'lstedu_bookinfo--teacher'}).find('strong').a.text
        sub.append(re.sub('[^가-힣]','',re.split(']', value)[0]))

    return(sub)

In [160]:
def get_id(lec_link):
    lec_id=[]
    tch_id=[]
    for i in lec_link:
        lec_id.append(re.split('&',re.split('=',lec_link[0])[1])[0])
        tch_id.append(re.split('=',lec_link[0])[3])
    return(lec_id,tch_id)

In [188]:
def get_complete(lec_link):
    complete=[]
    for i in lec_link:
        if re.split('&',re.split('=',i)[2])[0]=='1':
            complete.append('O')
        else:
            complete.append('X')
    return(complete)

In [110]:
driver = open_chromedriver()

In [42]:
driver.get('https://www.megastudy.net/')

In [43]:
open_popup()

In [44]:
change_frame()

In [45]:
switch_to_tch()

In [46]:
tch_home=get_tch_home()

In [47]:
page_url=get_page_url(tch_home)

In [48]:
### 기존 프레임으로 전환
driver.switch_to.parent_frame()

In [49]:
tch_name =[]
title = []
lec_link = []
num = 0
for i in page_url:
    num =num+1
    driver.get(i)
    
    time.sleep(1)
    
    try:
        select_lec()
    
        [x.extend(y) for x,y in zip([tch_name, title, lec_link], get_data_1())]
    except:
        pass
    
    print(num,'/',len(page_url))

1 / 103
2 / 103
3 / 103
4 / 103
5 / 103
6 / 103
7 / 103
8 / 103
9 / 103
10 / 103
11 / 103
12 / 103
13 / 103
14 / 103
15 / 103
16 / 103
17 / 103
18 / 103
19 / 103
20 / 103
21 / 103
22 / 103
23 / 103
24 / 103
25 / 103
26 / 103
27 / 103
28 / 103
29 / 103
30 / 103
31 / 103
32 / 103
33 / 103
34 / 103
35 / 103
36 / 103
37 / 103
38 / 103
39 / 103
40 / 103
41 / 103
42 / 103
43 / 103
44 / 103
45 / 103
46 / 103
47 / 103
48 / 103
49 / 103
50 / 103
51 / 103
52 / 103
53 / 103
54 / 103
55 / 103
56 / 103
57 / 103
58 / 103
59 / 103
60 / 103
61 / 103
62 / 103
63 / 103
64 / 103
65 / 103
66 / 103
67 / 103
68 / 103
69 / 103
70 / 103
71 / 103
72 / 103
73 / 103
74 / 103
75 / 103
76 / 103
77 / 103
78 / 103
79 / 103
80 / 103
81 / 103
82 / 103
83 / 103
84 / 103
85 / 103
86 / 103
87 / 103
88 / 103
89 / 103
90 / 103
91 / 103
92 / 103
93 / 103
94 / 103
95 / 103
96 / 103
97 / 103
98 / 103
99 / 103
100 / 103
101 / 103
102 / 103
103 / 103


In [142]:
sub = get_sub_tch_id(lec_link)

1 / 2224
2 / 2224
3 / 2224
4 / 2224
5 / 2224
6 / 2224
7 / 2224
8 / 2224
9 / 2224
10 / 2224
11 / 2224
12 / 2224
13 / 2224
14 / 2224
15 / 2224
16 / 2224
17 / 2224
18 / 2224
19 / 2224
20 / 2224
21 / 2224
22 / 2224
23 / 2224
24 / 2224
25 / 2224
26 / 2224
27 / 2224
28 / 2224
29 / 2224
30 / 2224
31 / 2224
32 / 2224
33 / 2224
34 / 2224
35 / 2224
36 / 2224
37 / 2224
38 / 2224
39 / 2224
40 / 2224
41 / 2224
42 / 2224
43 / 2224
44 / 2224
45 / 2224
46 / 2224
47 / 2224
48 / 2224
49 / 2224
50 / 2224
51 / 2224
52 / 2224
53 / 2224
54 / 2224
55 / 2224
56 / 2224
57 / 2224
58 / 2224
59 / 2224
60 / 2224
61 / 2224
62 / 2224
63 / 2224
64 / 2224
65 / 2224
66 / 2224
67 / 2224
68 / 2224
69 / 2224
70 / 2224
71 / 2224
72 / 2224
73 / 2224
74 / 2224
75 / 2224
76 / 2224
77 / 2224
78 / 2224
79 / 2224
80 / 2224
81 / 2224
82 / 2224
83 / 2224
84 / 2224
85 / 2224
86 / 2224
87 / 2224
88 / 2224
89 / 2224
90 / 2224
91 / 2224
92 / 2224
93 / 2224
94 / 2224
95 / 2224
96 / 2224
97 / 2224
98 / 2224
99 / 2224
100 / 2224
101 / 22

756 / 2224
757 / 2224
758 / 2224
759 / 2224
760 / 2224
761 / 2224
762 / 2224
763 / 2224
764 / 2224
765 / 2224
766 / 2224
767 / 2224
768 / 2224
769 / 2224
770 / 2224
771 / 2224
772 / 2224
773 / 2224
774 / 2224
775 / 2224
776 / 2224
777 / 2224
778 / 2224
779 / 2224
780 / 2224
781 / 2224
782 / 2224
783 / 2224
784 / 2224
785 / 2224
786 / 2224
787 / 2224
788 / 2224
789 / 2224
790 / 2224
791 / 2224
792 / 2224
793 / 2224
794 / 2224
795 / 2224
796 / 2224
797 / 2224
798 / 2224
799 / 2224
800 / 2224
801 / 2224
802 / 2224
803 / 2224
804 / 2224
805 / 2224
806 / 2224
807 / 2224
808 / 2224
809 / 2224
810 / 2224
811 / 2224
812 / 2224
813 / 2224
814 / 2224
815 / 2224
816 / 2224
817 / 2224
818 / 2224
819 / 2224
820 / 2224
821 / 2224
822 / 2224
823 / 2224
824 / 2224
825 / 2224
826 / 2224
827 / 2224
828 / 2224
829 / 2224
830 / 2224
831 / 2224
832 / 2224
833 / 2224
834 / 2224
835 / 2224
836 / 2224
837 / 2224
838 / 2224
839 / 2224
840 / 2224
841 / 2224
842 / 2224
843 / 2224
844 / 2224
845 / 2224
846 / 2224

1460 / 2224
1461 / 2224
1462 / 2224
1463 / 2224
1464 / 2224
1465 / 2224
1466 / 2224
1467 / 2224
1468 / 2224
1469 / 2224
1470 / 2224
1471 / 2224
1472 / 2224
1473 / 2224
1474 / 2224
1475 / 2224
1476 / 2224
1477 / 2224
1478 / 2224
1479 / 2224
1480 / 2224
1481 / 2224
1482 / 2224
1483 / 2224
1484 / 2224
1485 / 2224
1486 / 2224
1487 / 2224
1488 / 2224
1489 / 2224
1490 / 2224
1491 / 2224
1492 / 2224
1493 / 2224
1494 / 2224
1495 / 2224
1496 / 2224
1497 / 2224
1498 / 2224
1499 / 2224
1500 / 2224
1501 / 2224
1502 / 2224
1503 / 2224
1504 / 2224
1505 / 2224
1506 / 2224
1507 / 2224
1508 / 2224
1509 / 2224
1510 / 2224
1511 / 2224
1512 / 2224
1513 / 2224
1514 / 2224
1515 / 2224
1516 / 2224
1517 / 2224
1518 / 2224
1519 / 2224
1520 / 2224
1521 / 2224
1522 / 2224
1523 / 2224
1524 / 2224
1525 / 2224
1526 / 2224
1527 / 2224
1528 / 2224
1529 / 2224
1530 / 2224
1531 / 2224
1532 / 2224
1533 / 2224
1534 / 2224
1535 / 2224
1536 / 2224
1537 / 2224
1538 / 2224
1539 / 2224
1540 / 2224
1541 / 2224
1542 / 2224
1543

2143 / 2224
2144 / 2224
2145 / 2224
2146 / 2224
2147 / 2224
2148 / 2224
2149 / 2224
2150 / 2224
2151 / 2224
2152 / 2224
2153 / 2224
2154 / 2224
2155 / 2224
2156 / 2224
2157 / 2224
2158 / 2224
2159 / 2224
2160 / 2224
2161 / 2224
2162 / 2224
2163 / 2224
2164 / 2224
2165 / 2224
2166 / 2224
2167 / 2224
2168 / 2224
2169 / 2224
2170 / 2224
2171 / 2224
2172 / 2224
2173 / 2224
2174 / 2224
2175 / 2224
2176 / 2224
2177 / 2224
2178 / 2224
2179 / 2224
2180 / 2224
2181 / 2224
2182 / 2224
2183 / 2224
2184 / 2224
2185 / 2224
2186 / 2224
2187 / 2224
2188 / 2224
2189 / 2224
2190 / 2224
2191 / 2224
2192 / 2224
2193 / 2224
2194 / 2224
2195 / 2224
2196 / 2224
2197 / 2224
2198 / 2224
2199 / 2224
2200 / 2224
2201 / 2224
2202 / 2224
2203 / 2224
2204 / 2224
2205 / 2224
2206 / 2224
2207 / 2224
2208 / 2224
2209 / 2224
2210 / 2224
2211 / 2224
2212 / 2224
2213 / 2224
2214 / 2224
2215 / 2224
2216 / 2224
2217 / 2224
2218 / 2224
2219 / 2224
2220 / 2224
2221 / 2224
2222 / 2224
2223 / 2224
2224 / 2224


In [50]:
data_2 = get_data_2(lec_link)

https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48771&MAKE_FLG=2&tec_cd=megabori
1 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48772&MAKE_FLG=2&tec_cd=megabori
2 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48825&MAKE_FLG=2&tec_cd=megabori
3 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48826&MAKE_FLG=2&tec_cd=megabori
4 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48837&MAKE_FLG=2&tec_cd=megabori
5 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48838&MAKE_FLG=2&tec_cd=megabori
6 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48839&MAKE_FLG=2&tec_cd=megabori
7 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48409&MAKE_FLG=2&tec_cd=megabori
8 / 2224
https://www.mega

https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/lecture_detailview.asp?CHR_CD=40508&MAKE_FLG=1&tec_cd=ksk814
66 / 2224
https://www.megastudy.net/lecmain/mainh/chr_detail/naesin_detailview.asp?CHR_CD=36790&MAKE_FLG=1&tec_cd=ksk814
67 / 2224
https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/lecture_detailview.asp?CHR_CD=32431&MAKE_FLG=1&tec_cd=ksk814
68 / 2224
https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/lecture_detailview.asp?CHR_CD=32430&MAKE_FLG=1&tec_cd=ksk814
69 / 2224
https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/lecture_detailview.asp?CHR_CD=33681&MAKE_FLG=1&tec_cd=ksk814
70 / 2224
https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/lecture_detailview.asp?CHR_CD=32642&MAKE_FLG=1&tec_cd=ksk814
71 / 2224
https://www.megastudy.net/lecmain/mainh/chr_detail/naesin_detailview.asp?CHR_CD=43142&MAKE_FLG=1&tec_cd=ksk814
72 / 2224
https://www.megastudy.net/lecmain/mainh/chr_detail/naesin_detailview.asp?CHR_CD=43052&MAKE_FLG=1&t

https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=45402&MAKE_FLG=1&tec_cd=megakdw
132 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=45395&MAKE_FLG=1&tec_cd=megakdw
133 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=43312&MAKE_FLG=1&tec_cd=megakdw
134 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=43313&MAKE_FLG=1&tec_cd=megakdw
135 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=43314&MAKE_FLG=1&tec_cd=megakdw
136 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=43315&MAKE_FLG=1&tec_cd=megakdw
137 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=43316&MAKE_FLG=1&tec_cd=megakdw
138 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=43317&MAKE_FLG=1&tec_cd=megakdw
139 / 2224
https://

https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/lecture_detailview.asp?CHR_CD=48583&MAKE_FLG=1&tec_cd=rinassam1108
198 / 2224
https://www.megastudy.net/lecmain/mainh/chr_detail/naesin_detailview.asp?CHR_CD=46998&MAKE_FLG=1&tec_cd=rinassam1108
199 / 2224
https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/lecture_detailview.asp?CHR_CD=46999&MAKE_FLG=1&tec_cd=rinassam1108
200 / 2224
https://www.megastudy.net/lecmain/mainh/chr_detail/naesin_detailview.asp?CHR_CD=48309&MAKE_FLG=1&tec_cd=rinassam1108
201 / 2224
https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/lecture_detailview.asp?CHR_CD=48310&MAKE_FLG=1&tec_cd=rinassam1108
202 / 2224
https://www.megastudy.net/lecmain/mainh/chr_detail/naesin_detailview.asp?CHR_CD=47415&MAKE_FLG=1&tec_cd=rinassam1108
203 / 2224
https://www.megastudy.net/lecmain/mainh/chr_detail/naesin_detailview.asp?CHR_CD=47416&MAKE_FLG=1&tec_cd=rinassam1108
204 / 2224
https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/lecture_de

https://www.megastudy.net/lecmain/mainh/chr_detail/naesin_detailview.asp?CHR_CD=43451&MAKE_FLG=1&tec_cd=yw1122
261 / 2224
https://www.megastudy.net/lecmain/mainh/chr_detail/naesin_detailview.asp?CHR_CD=48026&MAKE_FLG=1&tec_cd=yw1122
262 / 2224
https://www.megastudy.net/lecmain/mainh/chr_detail/naesin_detailview.asp?CHR_CD=48264&MAKE_FLG=1&tec_cd=yw1122
263 / 2224
https://www.megastudy.net/lecmain/mainh/chr_detail/naesin_detailview.asp?CHR_CD=48265&MAKE_FLG=1&tec_cd=yw1122
264 / 2224
https://www.megastudy.net/lecmain/mainh/chr_detail/naesin_detailview.asp?CHR_CD=46461&MAKE_FLG=1&tec_cd=yw1122
265 / 2224
https://www.megastudy.net/lecmain/mainh/chr_detail/naesin_detailview.asp?CHR_CD=46573&MAKE_FLG=1&tec_cd=yw1122
266 / 2224
https://www.megastudy.net/lecmain/mainh/chr_detail/naesin_detailview.asp?CHR_CD=46574&MAKE_FLG=1&tec_cd=yw1122
267 / 2224
https://www.megastudy.net/lecmain/mainh/chr_detail/naesin_detailview.asp?CHR_CD=46622&MAKE_FLG=1&tec_cd=yw1122
268 / 2224
https://www.megastudy.ne

...

https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=46987&MAKE_FLG=1&tec_cd=psdm
1937 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48760&MAKE_FLG=1&tec_cd=psdm
1938 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=46787&MAKE_FLG=1&tec_cd=psdm
1939 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48734&MAKE_FLG=1&tec_cd=psdm
1940 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=46981&MAKE_FLG=1&tec_cd=psdm
1941 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48701&MAKE_FLG=1&tec_cd=psdm
1942 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=46741&MAKE_FLG=1&tec_cd=psdm
1943 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48686&MAKE_FLG=1&tec_cd=psdm
1944 / 2224
https://www.megastudy.ne

https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=44275&MAKE_FLG=1&tec_cd=principiabke
2003 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=44392&MAKE_FLG=1&tec_cd=principiabke
2004 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=30608&MAKE_FLG=1&tec_cd=principiabke
2005 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48630&MAKE_FLG=1&tec_cd=nonchujjs
2006 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48467&MAKE_FLG=1&tec_cd=nonchujjs
2007 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48044&MAKE_FLG=1&tec_cd=nonchujjs
2008 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=47925&MAKE_FLG=1&tec_cd=nonchujjs
2009 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=43196&MAKE_FLG=1&tec_

https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=46501&MAKE_FLG=1&tec_cd=memgacih
2068 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=46807&MAKE_FLG=1&tec_cd=memgacih
2069 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=46809&MAKE_FLG=1&tec_cd=memgacih
2070 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=45008&MAKE_FLG=1&tec_cd=memgacih
2071 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=45006&MAKE_FLG=1&tec_cd=memgacih
2072 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=45004&MAKE_FLG=1&tec_cd=memgacih
2073 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48222&MAKE_FLG=1&tec_cd=memgacih
2074 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48223&MAKE_FLG=1&tec_cd=memgacih
2075

https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=47152&MAKE_FLG=1&tec_cd=qlcsk9581
2133 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=43385&MAKE_FLG=1&tec_cd=qlcsk9581
2134 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=43270&MAKE_FLG=1&tec_cd=qlcsk9581
2135 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=43991&MAKE_FLG=1&tec_cd=qlcsk9581
2136 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48766&MAKE_FLG=2&tec_cd=megaggoarimy
2137 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48767&MAKE_FLG=2&tec_cd=megaggoarimy
2138 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48746&MAKE_FLG=2&tec_cd=megaggoarimy
2139 / 2224
https://www.megastudy.net/lecmain/mains/chr_detail/lecture_detailview.asp?CHR_CD=48747&MAKE_FLG=2&tec_

https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/lecture_detailview.asp?CHR_CD=48356&MAKE_FLG=1&tec_cd=mp1204
2196 / 2224
https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/lecture_detailview.asp?CHR_CD=47059&MAKE_FLG=1&tec_cd=mp1204
2197 / 2224
https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/lecture_detailview.asp?CHR_CD=46418&MAKE_FLG=1&tec_cd=mp1204
2198 / 2224
https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/lecture_detailview.asp?CHR_CD=47803&MAKE_FLG=1&tec_cd=mp1204
2199 / 2224
https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/lecture_detailview.asp?CHR_CD=47031&MAKE_FLG=1&tec_cd=mp1204
2200 / 2224
https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/lecture_detailview.asp?CHR_CD=47472&MAKE_FLG=1&tec_cd=mp1204
2201 / 2224
https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/lecture_detailview.asp?CHR_CD=47474&MAKE_FLG=1&tec_cd=mp1204
2202 / 2224
https://www.megastudy.net/lecmain/mains/mainsgo12/chr_detail/l

In [161]:
lec_id,tch_id=get_id(lec_link)

In [190]:
complete=get_complete(lec_link)

In [193]:
data = pd.DataFrame({'사이트':['메가스터디']*len(sub),'과목':sub,'선생님':tch_name,'선생님ID':tch_id,'강좌명':title,'강좌ID':lec_id,'완강여부':complete,'강의번호&챕터명&시간':data_2,'링크':lec_link})